In [ ]:
'''

conda environment 

train/validate/test

estimated long axis images (png file)

Deep learning models
1. 4 layer Conv BatchNorm ReLu CNN model
2. Transfer learning with EfficientNet-B0 pretrained net 

'''

# import pickle
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
import imageio
import skimage
import random
import tensorflow as tf
import PyQt5
%matplotlib qt

from glob import glob
from PIL import Image
from tensorflow import keras

from skimage import io, exposure
from shutil import copyfile, copytree
from scipy import ndimage

from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import EfficientNetB0, MobileNet, NASNetMobile, ResNet50, VGG16

import keras.backend as K

from sklearn.utils import class_weight  
from dl_func_la import gen_cnn_model, grayscale_to_rgb, resize_to_224, resize_to_96

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

gpu_no = '/gpu:0'

base_dir   = r'D:\python\yoonckim\longaxis'
excel_file = r'label_motion_10_bmcR1.xlsx'
dir_kfold  = r'data\CAT_KAG_kfold'


''' 
 model_name:
 '4_layer': CBR CNN with four layers 
 'EfficientNetB0': transfer learning (EfficientNetB0)
'''
# model_name = 'CustomCBR'
# model_name = 'EfficientNetB0'
# model_name = 'MobileNet'
# model_name = 'NASNetMobile'
# model_name = 'ResNet50'
model_name = 'VGG16'

# augment_method = 'noAugment'
augment_method = 'flipLR'  # 'flipLR' only

model_path = os.path.join('data/CAT_KAG_model1', model_name, augment_method)

fname1 = os.path.join(base_dir, excel_file)
df1 = pd.read_excel(fname1, sheet_name='Sheet1', index_col=None, engine='openpyxl')
print(df1['ID'].size)
df2 = df1[df1['image_interpretable'] == 'interpretable'] 
print(df2['ID'].size)

a = list(df2.columns)
col_list = a
print(col_list)

id_list = list(df2['ID'])
png_fname_list = list(df2['png_fname'])
isTrain_list = list(df2['isTrain'])
motion_flag_list = list(df2['motion_flag'])
    
# print(motion_flag_list)
png_train_dir = r'data\CAT_KAG\train'

nfold = 5


In [ ]:

if False:

    png_train_dir_CAT = r'data\CAT\train';
    png_train_dir_KAG = r'data\KAG_2ndAnnual\png\train';
    
    png_test_dir_CAT  = r'data\CAT\test'
    png_test_dir_KAG  = r'data\KAG_2ndAnnual\png\test';

    png_new_train_dir = r'data\CAT_KAG\train'
    png_new_test_dir  = r'data\CAT_KAG\test'
    
    isShuffling = False
    
    if isShuffling==True:
        list_dir1 = glob(png_train_dir + '\\DET0*')
        list_dir2 = glob(png_test_dir + '\\DET0*')
        list_dir3 = list_dir1 + list_dir2
        # print(list_dir3)
        list_dir3_rand = copy.copy(list_dir3)
        random.seed(870723)
        random.shuffle(list_dir3_rand)
        # print(list_dir3_rand)
        print(len(list_dir3_rand))
        ntrain = np.round(0.7 * len(list_dir3_rand)).astype(int)
        print(ntrain)
        list_dir_train = list_dir3_rand[:ntrain]
        list_dir_test = list_dir3_rand[ntrain:]
    else:
        list_dir1 = glob(png_train_dir_CAT + '\\DET0*')
        list_dir2 = glob(png_train_dir_KAG + '\\*')
        list_dir_train = list_dir1 + list_dir2
        
        list_dir1 = glob(png_test_dir_CAT + '\\DET0*')
        list_dir2 = glob(png_test_dir_KAG + '\\*')
        list_dir_test = list_dir1 + list_dir2       
        
    for jj, dir1 in enumerate(list_dir_train):
        dir_, id_folder = os.path.split(dir1)
        subj_png_dir = os.path.join(png_new_train_dir, id_folder)
        copytree(dir1, subj_png_dir)  
    
    for jj, dir1 in enumerate(list_dir_test):
        dir_, id_folder = os.path.split(dir1)
        subj_png_dir = os.path.join(png_new_test_dir, id_folder)
        copytree(dir1, subj_png_dir)  


In [ ]:
'''
create directories of five-folds data
'''

if False:
    list_dirs = glob(png_train_dir + '\\*')
    nsubj = len(list_dirs)
    print(f'nsubj = {nsubj}')

    list_dirs_rand = copy.copy(list_dirs)
    print(list_dirs)
    random.seed(870723)
    random.shuffle(list_dirs_rand)
    print(list_dirs_rand)

    fold_indices = []
    dir_fold = []
    nchunk = int(nsubj/nfold)

    for k in range(nfold):
        indices = np.arange(k*nchunk, (k+1)*nchunk)
        fold_indices.append(indices)

    # print(fold_indices[4])
    # print(0 in fold_indices[1])
    # print(list_dirs_rand[0])

    dir_, id_ = os.path.split(list_dirs_rand[0])
    # print(dir_)

    for kk, dir_name in enumerate(list_dirs_rand):
        for fold_group in range(nfold):
            if kk in fold_indices[fold_group]:
                dir_, id_folder = os.path.split(dir_name)
                subj_png_dir2 =  os.path.join(dir_kfold, 'fold'+str(fold_group+1), id_folder)
                dir_name2 = os.path.join(dir_name, 'estLA')
                copytree(dir_name2, subj_png_dir2)  
    

In [ ]:
''' 
functions

'''

def find_index_string(png_fname_list1, substring):
    
    for ind, fname_ in enumerate(png_fname_list1):
        if substring in fname_:
            index = ind
            break
        else:
            index = -1000 # in case not found
    
    return index

def scale_color(pixel_array):
    p2, p98 = np.percentile(pixel_array, (2, 98))
    
    return exposure.rescale_intensity(pixel_array, in_range=(p2, p98))

def thresh1(img):
    img_th = scale_color(img)
    
    return img_th


In [ ]:
class_num = 2   

history_dict = {}

val_fold_list = [1, 2, 3, 4, 5]
# val_fold_list = [3, 4, 5]
# val_fold_list = [5]

for _, val_f in enumerate(val_fold_list):

    if   val_f==1:        train_fold = [2,3,4,5]         
    elif val_f==2:        train_fold = [1,3,4,5]         
    elif val_f==3:        train_fold = [1,2,4,5]         
    elif val_f==4:        train_fold = [1,2,3,5]         
    elif val_f==5:        train_fold = [1,2,3,4]         
    val_fold = [val_f]
    
    print(f'training data: val fold = {val_f}')
    
    train_img    = list()
    train_motion = list()

    val_img      = list()
    val_motion   = list()

    for jj, value in enumerate(train_fold):
        dir_3 = os.path.join(dir_kfold, 'fold'+str(value))
        dir_id = glob(dir_3 + '\\*')

        for kk, id_ in enumerate(dir_id):
            list_png = glob(id_ + '\\*.png')       
            
            for ll, fname_png in enumerate(list_png):
                dir_, fname_ = os.path.split(fname_png)
                img = imageio.imread(fname_png) 
                img1 = skimage.transform.resize(img, (96, 256), anti_aliasing=True, preserve_range=False)
                
                # crop and normalize
                img2 = img1[:, (128-96):(128+96)]
                sh = 10 # pixel
                img2m20t = img1[:, (128-sh-96):(128-sh+96)]
                img2p20t = img1[:, (128+sh-96):(128+sh+96)]
                
                img2 = img2/np.max(img2)
                ang = 5 # degree
                img2m10r = ndimage.rotate(img2, -ang, reshape=False)
                img2p10r = ndimage.rotate(img2,  ang, reshape=False)
                
                img2m20t = img2m20t/np.max(img2m20t)
                img2p20t = img2p20t/np.max(img2p20t)               
                
                ind = find_index_string(png_fname_list, fname_)
                if ind == -1000:
                    print(f'{fname_} is outlier. continue')
                    continue

                if 'presen' in motion_flag_list[ind]: 
                    motion_eLA = 1
                elif 'absen' in motion_flag_list[ind]: 
                    motion_eLA = 0
                else: quit()    
                
                if augment_method == 'aug_method3':
                    #print(f'{augment_method} performed!')
                    train_img.append(img2);                  train_motion.append(motion_eLA)
                    train_img.append(np.fliplr(img2));       train_motion.append(motion_eLA)
                    train_img.append(img2m10r);              train_motion.append(motion_eLA)
                    train_img.append(np.fliplr(img2m10r));   train_motion.append(motion_eLA)
                    train_img.append(img2p10r);              train_motion.append(motion_eLA)
                    train_img.append(np.fliplr(img2p10r));   train_motion.append(motion_eLA)
                    train_img.append(img2m20t);              train_motion.append(motion_eLA)
                    train_img.append(np.fliplr(img2m20t));   train_motion.append(motion_eLA)
                    train_img.append(img2p20t);              train_motion.append(motion_eLA)
                    train_img.append(np.fliplr(img2p20t));   train_motion.append(motion_eLA)
                    
                    if jj==0 and kk==1 and ll==0 and False:
                        plt.figure(figsize=(12, 5))
                        plt.subplot(251)
                        plt.imshow(img2, cmap='gray'); plt.title('img1: img1'); plt.axis('off')
                        plt.subplot(252)
                        plt.imshow(img2m10r, cmap='gray'); plt.title('img2: -10-deg rotate'); plt.axis('off')
                        plt.subplot(253)
                        plt.imshow(img2p10r, cmap='gray'); plt.title('img3: +10-deg rotate'); plt.axis('off')
                        plt.subplot(254)
                        plt.imshow(img2m20t, cmap='gray'); plt.title('img4: -20 pix translate'); plt.axis('off')
                        plt.subplot(255)
                        plt.imshow(img2p20t, cmap='gray'); plt.title('img5: +20 pix translate'); plt.axis('off')
                        
                        plt.subplot(256)
                        plt.imshow(np.fliplr(img2), cmap='gray'); plt.title('img6: fliplr of img1'); plt.axis('off')
                        plt.subplot(257)
                        plt.imshow(np.fliplr(img2m10r), cmap='gray'); plt.title('img7: fliplr of img2'); plt.axis('off')
                        plt.subplot(258)
                        plt.imshow(np.fliplr(img2p10r), cmap='gray'); plt.title('img8: fliplr of img3'); plt.axis('off')
                        plt.subplot(259)
                        plt.imshow(np.fliplr(img2m20t), cmap='gray'); plt.title('img9: fliplr of img4'); plt.axis('off')
                        plt.subplot(2,5,10)
                        plt.imshow(np.fliplr(img2p20t), cmap='gray'); plt.title('img10: fliplr of img5'); plt.axis('off')
                        
                        plt.show()
                    
                elif augment_method == 'aug_method2':
                    #print(f'{augment_method} performed!')
                    train_img.append(img2);            train_motion.append(motion_eLA)
                    train_img.append(img2m10r);        train_motion.append(motion_eLA)
                    train_img.append(img2p10r);        train_motion.append(motion_eLA)
                    train_img.append(img2m20t);        train_motion.append(motion_eLA)
                    train_img.append(img2p20t);        train_motion.append(motion_eLA)
                    
                elif augment_method == 'flipLR':
                    train_img.append(img2);            train_motion.append(motion_eLA)
                    train_img.append(np.fliplr(img2)); train_motion.append(motion_eLA)
                    
                elif augment_method == 'noAugment':  
                    #print('No Augment')
                    train_img.append(img2);            train_motion.append(motion_eLA)
                
    print(len(train_img), len(train_motion))
    
    for jj, value in enumerate(val_fold):
        dir_3 = os.path.join(dir_kfold, 'fold'+str(value))
        dir_id = glob(dir_3 + '\\*')
        print(len(dir_id))

        for kk, id_ in enumerate(dir_id):
            list_png = glob(id_+'\\*.png')       

            for ll, fname_png in enumerate(list_png):
                dir_, fname_ = os.path.split(fname_png)
                img = imageio.imread(fname_png)
                img2 = skimage.transform.resize(img, (96, 256), anti_aliasing=True, preserve_range=False)
                
                img2 = img2[:, (128-96):(128+96)]
                img2 = img2/np.max(img2)
                
                ind = find_index_string(png_fname_list, fname_)
                if ind == -1000:
                    print(f'{fname_} is outlier. continue')
                    continue                
                
                if 'presen' in motion_flag_list[ind]: 
                    motion_eLA = 1
                elif 'absen' in motion_flag_list[ind]: 
                    motion_eLA = 0
                else: quit()
                
                val_img.append(img2)
                val_motion.append(motion_eLA)

    print(len(val_img), len(val_motion))

    X_train = np.stack(train_img, axis=0)
    y_train = np.stack(train_motion, axis=0)

    X_val = np.stack(val_img, axis=0)
    y_val = np.stack(val_motion, axis=0)

    X_train_ = grayscale_to_rgb(X_train)
    X_val_ = grayscale_to_rgb(X_val)

    sess2 = tf.compat.v1.Session()
    with sess2.as_default():
        X_train = X_train_.numpy()
        X_val = X_val_.numpy()

    print(f'X_train shape: {X_train.shape}, X_val shape: {X_val.shape}')
        
    #
    # check class imbalance
    #

    #### inter-breath-hold motion detection
    print('inter-breath-hold motion detection')
    pos = np.where(y_train==1)[0];     n_pos = len(pos)
    print(f'train     : motion(+) group: # of images = {n_pos}')
    
    neg = np.where(y_train==0)[0];    n_neg = len(neg)
    print(f'train     : motion(-) group: # of images = {n_neg}')
    
    pos = np.where(y_val==1)[0];    n_pos = len(pos)
    print(f'validation: motion(+) group: # of images = {n_pos}')
    
    neg = np.where(y_val==0)[0];    n_neg = len(neg)
    print(f'validation: motion(-) group: # of images = {n_neg}')

    input_shape_TLmodel = '224'
    
    if model_name == 'CustomCBR':
    
        n_filter = 16; 
        kernel1 = (3, 3); 
        padding1 = 'same' 
        strides1 = (1, 1)
        input_shape = (96, 192, 3)
        pool_size1 = (2, 2)
        pool_size2 = (2, 2)

        learning_rate = 0.00001    #0.000001  #0.000001  #0.00001  #0.000001
        batch_size = 4
        n_epochs = 50 # 200 #50 #100 #150

        with K.tf.device(gpu_no):

            model = Sequential()
            model.add(Conv2D(filters=n_filter, kernel_size=kernel1, padding=padding1, strides=strides1, input_shape=input_shape))
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=pool_size2) )

            model.add(Conv2D(filters=2*n_filter, kernel_size=kernel1, padding=padding1, strides=strides1) )
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=pool_size1) )         

            model.add(Conv2D(filters=4*n_filter, kernel_size=kernel1, padding=padding1, strides=strides1))
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=pool_size1)  )   

            model.add(Conv2D(filters=8*n_filter, kernel_size=kernel1, padding=padding1, strides=strides1))
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=pool_size1)  )

            model.add(Flatten()) 
            model.add(Dense(64))
            model.add(Activation("relu"))

            model.add(Dropout(0.5))

            model.add(Dense(class_num, activation='sigmoid'))
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
        
        # plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

        # prepare data for CNN
        if False:
            for ind in range(2, y_train.shape[0], 30):    
                label = y_train[ind]
                plt.figure(figsize=(9, 3))
                img1 = X_train[ind, :, :, 0]
                plt.imshow(img1, cmap='gray')

                if class_num == 2:
                    plt.title(f'label={label}: motion=1, no motion=0')

            plt.show()

    elif model_name == 'EfficientNetB0':
        
        input_shape_TLmodel = '224'
                        
        # input_shape = (96, 128, 3) => (224, 224, 3) shape conversion
        if input_shape_TLmodel == '224':
            learning_rate = 0.0001  # 0.0001
            input_shape = (224, 224, 3)
            inputs = Input(shape=input_shape)
                
            X_train2 = resize_to_224(X_train) 
            X_val2 = resize_to_224(X_val)
        
        elif input_shape_TLmodel == '96':
            learning_rate = 0.0005  # 0.0001
            input_shape = (96, 128, 3)
            inputs = Input(shape=input_shape)
            
            X_train2 = resize_to_96(X_train)
            X_val2 = resize_to_96(X_val)
            
        batch_size = 2  
        n_epochs = 50
        
        x = inputs
        
        with K.tf.device(gpu_no):
            model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")
            # Freeze the pretrained weights
            model.trainable = False

            # Rebuild top
            x = GlobalAveragePooling2D(name="avg_pool")(model.output)
            x = BatchNormalization()(x)

            top_dropout_rate = 0.2
            x = Dropout(top_dropout_rate, name="top_dropout")(x)
            outputs = Dense(class_num, activation="softmax", name="pred")(x)

            # Compile
            model = tf.keras.Model(inputs, outputs, name="EfficientNet")
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])
    
            #epochs = 50  # @param {type: "slider", min:8, max:80}
            #history1 = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)
            #plot_hist(hist)    
        
    elif model_name == 'MobileNet':
        
        input_shape_TLmodel = '224'
                
        if input_shape_TLmodel == '224':
            learning_rate = 0.0001  # 0.0001
            input_shape = (224, 224, 3)
            inputs = Input(shape=input_shape)
                
            X_train2 = resize_to_224(X_train) 
            X_val2 = resize_to_224(X_val)
        
        batch_size = 2  
        n_epochs = 50
        
        x = inputs
        
        with K.tf.device(gpu_no):
            model = MobileNet(include_top=False, input_tensor=x, weights="imagenet")
            # Freeze the pretrained weights
            model.trainable = False

            # Rebuild top
            x = GlobalAveragePooling2D(name="avg_pool")(model.output)
            x = BatchNormalization()(x)

            top_dropout_rate = 0.2
            x = Dropout(top_dropout_rate, name="top_dropout")(x)
            outputs = Dense(class_num, activation="softmax", name="pred")(x)

            # Compile
            model = tf.keras.Model(inputs, outputs, name="MobileNet")
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])

    elif model_name == 'NASNetMobile':
        
        input_shape_TLmodel = '224'

        if input_shape_TLmodel == '224':
            learning_rate = 0.0001  # 0.0001
            input_shape = (224, 224, 3)
            inputs = Input(shape=input_shape)
                
            X_train2 = resize_to_224(X_train) 
            X_val2 = resize_to_224(X_val)
        
        batch_size = 2 
        n_epochs = 50
        
        x = inputs
        
        with K.tf.device(gpu_no):
            model = NASNetMobile(include_top=False, input_tensor=x, weights="imagenet")
            # Freeze the pretrained weights
            model.trainable = False

            # Rebuild top
            x = GlobalAveragePooling2D(name="avg_pool")(model.output)
            x = BatchNormalization()(x)

            top_dropout_rate = 0.2
            x = Dropout(top_dropout_rate, name="top_dropout")(x)
            outputs = Dense(class_num, activation="softmax", name="pred")(x)

            # Compile
            model = tf.keras.Model(inputs, outputs, name="NASNetMobile")
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])
    

    elif model_name == 'ResNet50':
        
        input_shape_TLmodel = '224'
                
        if input_shape_TLmodel == '224':
            learning_rate = 0.0001  # 0.0001
            input_shape = (224, 224, 3)
            inputs = Input(shape=input_shape)
                
            X_train2 = resize_to_224(X_train) 
            X_val2 = resize_to_224(X_val)
        
        batch_size = 2  
        n_epochs = 50
        
        x = inputs
        
        with K.tf.device(gpu_no):
            model = ResNet50(include_top=False, input_tensor=x, weights="imagenet")
            # Freeze the pretrained weights
            model.trainable = False

            # Rebuild top
            x = GlobalAveragePooling2D(name="avg_pool")(model.output)
            x = BatchNormalization()(x)

            top_dropout_rate = 0.2
            x = Dropout(top_dropout_rate, name="top_dropout")(x)
            outputs = Dense(class_num, activation="softmax", name="pred")(x)

            # Compile
            model = tf.keras.Model(inputs, outputs, name="ResNet50")
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])
    
    elif model_name == 'VGG16':
        
        input_shape_TLmodel = '224'
        if input_shape_TLmodel == '224':
            learning_rate = 0.0001  # 0.0001
            input_shape = (224, 224, 3)
            inputs = Input(shape=input_shape)
                
            X_train2 = resize_to_224(X_train) 
            X_val2 = resize_to_224(X_val)
        
        batch_size = 2  
        n_epochs = 50
        
        x = inputs
        
        with K.tf.device(gpu_no):
            model = VGG16(include_top=False, input_tensor=x, weights="imagenet")
            # Freeze the pretrained weights
            model.trainable = False

            # Rebuild top
            x = GlobalAveragePooling2D(name="avg_pool")(model.output)
            x = BatchNormalization()(x)

            top_dropout_rate = 0.2
            x = Dropout(top_dropout_rate, name="top_dropout")(x)
            outputs = Dense(class_num, activation="softmax", name="pred")(x)

            # Compile
            model = tf.keras.Model(inputs, outputs, name="VGG16")
            model.compile(optimizer=Adam(learning_rate=learning_rate), loss="binary_crossentropy", metrics=["accuracy"])
    
    print(X_train2.shape, X_val2.shape)
    history1 = gen_cnn_model(gpu_no, model, model_path, val_fold[0], X_train2, y_train, X_val2, y_val, batch_size, n_epochs)
    
    # plot training and validation accuracy/loss values

    history_list = list()
    history_list.append(history1)

    history_dict['val_fold='+str(val_f)] = history_list


In [ ]:
print(history_dict.keys())
print(val_fold[0])

for kk in [1,2,3,4,5]:
    hist_list = history_dict['val_fold='+str(kk)]
    print(len(hist_list))
    print(hist_list[0].history.keys())
    
    if kk==1:
        col = 'blue'
    elif kk==2:
        col = 'green'
    elif kk==3:
        col = 'red'
    elif kk==4:
        col = 'orange'
    elif kk==5:
        col = 'black'
    
    for ll, hist2 in enumerate(hist_list):
        
        plt.figure(num=1, figsize=(15, 8))
        
        plt.subplot(221)
        plt.plot(hist2.history['loss'], color=col)
        plt.title(f'Train loss')
        plt.ylabel('Loss'); plt.xlabel('Epoch')
        plt.ylim([0.0, 2.0])
        plt.grid(True)
        plt.legend(['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'], loc='upper right')
        
        plt.subplot(222)
        plt.plot(hist2.history['accuracy'], color=col)
        plt.title(f'Train accuracy')
        plt.ylabel('Accuracy'); plt.xlabel('Epoch')
        plt.ylim([0.2, 1.0])
        plt.grid(True)
        plt.legend(['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'], loc='upper left')
        
        plt.subplot(223)
        plt.plot(hist2.history['val_loss'], color=col)
        plt.title(f'Validation loss')
        plt.ylabel('Loss'); plt.xlabel('Epoch')
        plt.ylim([0.0, 2.0])
        plt.grid(True)
        plt.legend(['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'], loc='upper right')
        
        plt.subplot(224)
        plt.plot(hist2.history['val_accuracy'], color=col)
        plt.title(f'Validation accuracy')
        plt.ylabel('Accuracy'); plt.xlabel('Epoch')
        plt.ylim([0.2, 1.0])
        plt.grid(True)
        plt.legend(['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'], loc='upper left')